In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset.columns

Index(['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased'], dtype='object')

In [5]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [6]:
dataset = dataset.drop("User ID",axis=1)

In [7]:
inde = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dep = dataset[['Purchased']]

In [8]:
dataset['Purchased'].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
inde.shape

(400, 3)

In [11]:
dep.shape

(400, 1)

In [12]:
dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test =train_test_split(inde, dep, test_size = 1/3, random_state=0)

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid = {'kernel': ['rbf', 'poly','sigmoid','linear'],
             'C': [3000,5000,6000],'gamma':['auto','scale']}

grid =GridSearchCV(SVC(probability=True), param_grid, refit=True, verbose=3 ,n_jobs=-1 ,scoring = 'f1_weighted')

In [16]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\2155586\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [3000, 5000, 6000], 'gamma': ['auto', 'scale'],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             scoring='f1_weighted', verbose=3)

In [17]:
re = grid.cv_results_

grid_predictions= grid.predict(x_test)

In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,grid_predictions)

In [19]:
print(cm)

[[78  7]
 [ 6 43]]


In [20]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [21]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.93      0.92      0.92        85
           1       0.86      0.88      0.87        49

    accuracy                           0.90       134
   macro avg       0.89      0.90      0.90       134
weighted avg       0.90      0.90      0.90       134



In [22]:
from sklearn.metrics import f1_score
f1_macro=f1_score (y_test, grid_predictions, average= 'weighted' )
print("the f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)

the f1_macro value for best parameter{'C': 6000, 'gamma': 'scale', 'kernel': 'rbf'}: 0.9031880225910075


In [23]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9478991596638655

In [24]:
table=pd.DataFrame.from_dict(re)

In [25]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.192945,0.049443,0.025823,0.012795,3000,auto,rbf,"{'C': 3000, 'gamma': 'auto', 'kernel': 'rbf'}",0.826263,0.866968,0.851527,0.847020,0.883278,0.855011,0.019206,3
1,3.155027,1.203394,0.021058,0.012985,3000,auto,poly,"{'C': 3000, 'gamma': 'auto', 'kernel': 'poly'}",0.822092,0.765553,0.766556,0.924528,0.901744,0.836095,0.066541,7
2,0.022678,0.005737,0.019616,0.010794,3000,auto,sigmoid,"{'C': 3000, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.791752,0.714931,0.698113,0.796284,0.766556,0.753527,0.040048,22
3,2.126492,0.530079,0.021520,0.020625,3000,auto,linear,"{'C': 3000, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,13
4,0.241884,0.048616,0.028551,0.020122,3000,scale,rbf,"{'C': 3000, 'gamma': 'scale', 'kernel': 'rbf'}",0.826263,0.866968,0.851527,0.847020,0.883278,0.855011,0.019206,3
5,2.988802,1.549342,0.020908,0.014396,3000,scale,poly,"{'C': 3000, 'gamma': 'scale', 'kernel': 'poly'}",0.822092,0.765553,0.766556,0.924528,0.901744,0.836095,0.066541,7
6,0.034820,0.024916,0.028852,0.021360,3000,scale,sigmoid,"{'C': 3000, 'gamma': 'scale', 'kernel': 'sigmo...",0.808927,0.714931,0.698113,0.796284,0.766556,0.756962,0.043745,19
7,2.033855,0.499022,0.020835,0.020330,3000,scale,linear,"{'C': 3000, 'gamma': 'scale', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,13
8,0.261306,0.097988,0.022703,0.009632,5000,auto,rbf,"{'C': 5000, 'gamma': 'auto', 'kernel': 'rbf'}",0.826263,0.866968,0.851527,0.847020,0.883278,0.855011,0.019206,3
9,5.080480,2.846591,0.015284,0.005291,5000,auto,poly,"{'C': 5000, 'gamma': 'auto', 'kernel': 'poly'}",0.822092,0.765553,0.766556,0.924528,0.901744,0.836095,0.066541,7


In [32]:
future_prediction=grid.predict([[0,36,33000]])
print("future_predicition={}".format(future_prediction))

future_predicition=[1]


In [33]:
import sklearn
print(sklearn.__version__)

1.2.2


In [34]:
import sklearn
print('Scikit-learn version:', sklearn.__version__)

import scipy
print('SciPy version:', scipy.__version__)

import numpy
print('NumPy version:', numpy.__version__)

Scikit-learn version: 1.2.2
SciPy version: 1.10.1
NumPy version: 1.24.3


In [35]:
conda update -n base -c defaults conda

^C

Note: you may need to restart the kernel to use updated packages.


In [36]:
import sklearn
print('Scikit-learn version:', sklearn.__version__)

Scikit-learn version: 1.2.2
